In [1]:
import os
import sys
import spark_utils as sut
import pyspark.sql.functions as F

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

from pyspark.sql.types import StructType, StructField, IntegerType, StringType
import pyspark.sql.functions as F
spark = sut.get_spark_session()


# Introduction

This notebook demos the custom PySpark functions discussed in [Speed up Your ML Projects With Spark -- Handy Custom {pySpark} Functions (II)](https://medium.com/@menawang/speed-up-your-ml-projects-with-spark-675c5e269d13) published on [Towards AI](https://pub.towardsai.net/). 

The revelant functions were saved in [spark_utils.py](spark_utils.py) and imported into this notebook for demo by `import spark_utils as sut`. 

## Demo Spark DataFrame

You probably guessed it by looking at the dummy data below. In this article, we will discuss functions to handle tricky dates and messy numeric values.

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, col
from typing import List
from pyspark.sql import DataFrame

# Initialize Spark session
spark = SparkSession.builder.appName("SparkDemo").getOrCreate()

# Sample data with 'yyyy/MM/dd' date format
data = [
    ("A", "101", "15Jul2023", "2023/07/15", "2023-07-15 09:07:33.0", "QLD-individual", "130000"),
    ("B", "102", "20Aug2023", "2023/08/20", "2023-08-20 10:08:45.0", "NSW-business", "130000"),
    ("C", "103", "25Oct2023", "2023/10/25", "2023-10-25 11:10:50.0", "VIC-business", "95000"),
    ("A", "101", "15Jan2024", "2024/01/15", "2024-01-15 12:01:30.0", "TAS-individual", "110000"),
    ("B", "102", "19Feb2024", "2024/02/19", "2024-02-19 13:02:40.0", "NSW-business", "140000"),
    ("C", "103", "15Jan2024", "2024/01/15", "2024-01-15 14:01:55.0", "QLD-business", "115000")
]

# Create DataFrame
df = spark.createDataFrame(data, ["ID1", "ID2", "date_1", "date_2", "date_3", "description", "amount"])

df = df.orderBy('ID1','ID2')

# Show DataFrame
df.show()

+---+---+---------+----------+--------------------+--------------+------+
|ID1|ID2|   date_1|    date_2|              date_3|   description|amount|
+---+---+---------+----------+--------------------+--------------+------+
|  A|101|15Jan2024|2024/01/15|2024-01-15 12:01:...|TAS-individual|110000|
|  A|101|15Jul2023|2023/07/15|2023-07-15 09:07:...|QLD-individual|130000|
|  B|102|20Aug2023|2023/08/20|2023-08-20 10:08:...|  NSW-business|130000|
|  B|102|19Feb2024|2024/02/19|2024-02-19 13:02:...|  NSW-business|140000|
|  C|103|15Jan2024|2024/01/15|2024-01-15 14:01:...|  QLD-business|115000|
|  C|103|25Oct2023|2023/10/25|2023-10-25 11:10:...|  VIC-business| 95000|
+---+---+---------+----------+--------------------+--------------+------+



In [3]:
sut.print_schema_alphabetically(df)

root
 |-- ID1: string (nullable = true)
 |-- ID2: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- date_1: string (nullable = true)
 |-- date_2: string (nullable = true)
 |-- date_3: string (nullable = true)
 |-- description: string (nullable = true)



## transform date cols

In [4]:
df = sut.transform_date_cols(df,['date_1'])
df.show()

+---+---+----------+----------+--------------------+--------------+------+
|ID1|ID2|    date_1|    date_2|              date_3|   description|amount|
+---+---+----------+----------+--------------------+--------------+------+
|  A|101|2023-07-15|2023/07/15|2023-07-15 09:07:...|QLD-individual|130000|
|  A|101|2024-01-15|2024/01/15|2024-01-15 12:01:...|TAS-individual|110000|
|  B|102|2024-02-19|2024/02/19|2024-02-19 13:02:...|  NSW-business|140000|
|  B|102|2023-08-20|2023/08/20|2023-08-20 10:08:...|  NSW-business|130000|
|  C|103|2023-10-25|2023/10/25|2023-10-25 11:10:...|  VIC-business| 95000|
|  C|103|2024-01-15|2024/01/15|2024-01-15 14:01:...|  QLD-business|115000|
+---+---+----------+----------+--------------------+--------------+------+



In [5]:
df = sut.transform_date_cols(df,['date_2'], 
                             str_date_format='yyyy/MM/dd')
df.show()

+---+---+----------+----------+--------------------+--------------+------+
|ID1|ID2|    date_1|    date_2|              date_3|   description|amount|
+---+---+----------+----------+--------------------+--------------+------+
|  A|101|2023-07-15|2023-07-15|2023-07-15 09:07:...|QLD-individual|130000|
|  A|101|2024-01-15|2024-01-15|2024-01-15 12:01:...|TAS-individual|110000|
|  B|102|2023-08-20|2023-08-20|2023-08-20 10:08:...|  NSW-business|130000|
|  B|102|2024-02-19|2024-02-19|2024-02-19 13:02:...|  NSW-business|140000|
|  C|103|2024-01-15|2024-01-15|2024-01-15 14:01:...|  QLD-business|115000|
|  C|103|2023-10-25|2023-10-25|2023-10-25 11:10:...|  VIC-business| 95000|
+---+---+----------+----------+--------------------+--------------+------+



In [6]:
df = sut.transform_date_cols(df,['date_3'], 
                             str_date_format='yyyy-MM-dd HH:mm:ss.S')
df.show()

+---+---+----------+----------+----------+--------------+------+
|ID1|ID2|    date_1|    date_2|    date_3|   description|amount|
+---+---+----------+----------+----------+--------------+------+
|  A|101|2024-01-15|2024-01-15|2024-01-15|TAS-individual|110000|
|  A|101|2023-07-15|2023-07-15|2023-07-15|QLD-individual|130000|
|  B|102|2023-08-20|2023-08-20|2023-08-20|  NSW-business|130000|
|  B|102|2024-02-19|2024-02-19|2024-02-19|  NSW-business|140000|
|  C|103|2023-10-25|2023-10-25|2023-10-25|  VIC-business| 95000|
|  C|103|2024-01-15|2024-01-15|2024-01-15|  QLD-business|115000|
+---+---+----------+----------+----------+--------------+------+



In [7]:
sut.print_schema_alphabetically(df.select(['date_1','date_2', 'date_3']))

root
 |-- date_1: date (nullable = true)
 |-- date_2: date (nullable = true)
 |-- date_3: date (nullable = true)



In [8]:
# test function on a col that is already in date format
check = sut.transform_date_cols(df,['date_2'])
check.show()

+---+---+----------+----------+----------+--------------+------+
|ID1|ID2|    date_1|    date_2|    date_3|   description|amount|
+---+---+----------+----------+----------+--------------+------+
|  A|101|2024-01-15|2024-01-15|2024-01-15|TAS-individual|110000|
|  A|101|2023-07-15|2023-07-15|2023-07-15|QLD-individual|130000|
|  B|102|2023-08-20|2023-08-20|2023-08-20|  NSW-business|130000|
|  B|102|2024-02-19|2024-02-19|2024-02-19|  NSW-business|140000|
|  C|103|2024-01-15|2024-01-15|2024-01-15|  QLD-business|115000|
|  C|103|2023-10-25|2023-10-25|2023-10-25|  VIC-business| 95000|
+---+---+----------+----------+----------+--------------+------+



## filter by date

In [9]:
check = sut.filter_by_date(df, 'date_2', 
                           min_date="2023-07-01", 
                           max_date="2024-02-05",
                           original_date_format='yyyy/MM/dd')
check.show()

+---+---+----------+----------+----------+--------------+------+
|ID1|ID2|    date_1|    date_2|    date_3|   description|amount|
+---+---+----------+----------+----------+--------------+------+
|  A|101|2023-07-15|2023-07-15|2023-07-15|QLD-individual|130000|
|  A|101|2024-01-15|2024-01-15|2024-01-15|TAS-individual|110000|
|  B|102|2023-08-20|2023-08-20|2023-08-20|  NSW-business|130000|
|  C|103|2023-10-25|2023-10-25|2023-10-25|  VIC-business| 95000|
|  C|103|2024-01-15|2024-01-15|2024-01-15|  QLD-business|115000|
+---+---+----------+----------+----------+--------------+------+



## get distinct values

In [10]:
sut.get_distinct_values(check, 'date_1')

[datetime.date(2023, 7, 15),
 datetime.date(2023, 8, 20),
 datetime.date(2023, 10, 25),
 datetime.date(2024, 1, 15)]

## top_rows_for_IDs

In [11]:
# get the latest record for each ID1 & ID2 combination
check = sut.top_rows_for_ids(df,['ID1','ID2'], 
                            value_field = 'date_1', 
                            ascending = False)
check.show()

+---+---+----------+----------+----------+--------------+------+
|ID1|ID2|    date_1|    date_2|    date_3|   description|amount|
+---+---+----------+----------+----------+--------------+------+
|  A|101|2024-01-15|2024-01-15|2024-01-15|TAS-individual|110000|
|  B|102|2024-02-19|2024-02-19|2024-02-19|  NSW-business|140000|
|  C|103|2024-01-15|2024-01-15|2024-01-15|  QLD-business|115000|
+---+---+----------+----------+----------+--------------+------+



In [12]:
# get the record with the highest amount for each ID
check = sut.top_rows_for_ids(df,['ID1','ID2'], 
                            value_field = 'amount', 
                            ascending = False)
check.show()

+---+---+----------+----------+----------+--------------+------+
|ID1|ID2|    date_1|    date_2|    date_3|   description|amount|
+---+---+----------+----------+----------+--------------+------+
|  A|101|2023-07-15|2023-07-15|2023-07-15|QLD-individual|130000|
|  B|102|2024-02-19|2024-02-19|2024-02-19|  NSW-business|140000|
|  C|103|2023-10-25|2023-10-25|2023-10-25|  VIC-business| 95000|
+---+---+----------+----------+----------+--------------+------+



## clean dollar values

In [13]:
df = sut.clean_dollar_cols(df,['amount'])
df.show()

+---+---+----------+----------+----------+--------------+--------+
|ID1|ID2|    date_1|    date_2|    date_3|   description|  amount|
+---+---+----------+----------+----------+--------------+--------+
|  A|101|2024-01-15|2024-01-15|2024-01-15|TAS-individual|110000.0|
|  A|101|2023-07-15|2023-07-15|2023-07-15|QLD-individual|130000.0|
|  B|102|2024-02-19|2024-02-19|2024-02-19|  NSW-business|140000.0|
|  B|102|2023-08-20|2023-08-20|2023-08-20|  NSW-business|130000.0|
|  C|103|2024-01-15|2024-01-15|2024-01-15|  QLD-business|115000.0|
|  C|103|2023-10-25|2023-10-25|2023-10-25|  VIC-business| 95000.0|
+---+---+----------+----------+----------+--------------+--------+



## filter df by strings

In the example below, we can use the function to find `individual` records in either `QLD` or `NSW` in two steps.

In [14]:
# find `individual` records in either `QLD` or `NSW` in two steps.
print("---- Step 1: Filter for either states")
search_strings = ['QLD', "NSW"]
state_filtered = sut.filter_df_by_strings(df, 'description', search_strings)
state_filtered.show()
print("---- Step2: Find 'individual' records only")
final = sut.filter_df_by_strings(state_filtered, 'description', ['individual'])
final.show()

---- Step 1: Filter for either states
+---+---+----------+----------+----------+--------------+--------+
|ID1|ID2|    date_1|    date_2|    date_3|   description|  amount|
+---+---+----------+----------+----------+--------------+--------+
|  A|101|2023-07-15|2023-07-15|2023-07-15|QLD-individual|130000.0|
|  B|102|2023-08-20|2023-08-20|2023-08-20|  NSW-business|130000.0|
|  B|102|2024-02-19|2024-02-19|2024-02-19|  NSW-business|140000.0|
|  C|103|2024-01-15|2024-01-15|2024-01-15|  QLD-business|115000.0|
+---+---+----------+----------+----------+--------------+--------+

---- Step2: Find 'individual' records only
+---+---+----------+----------+----------+--------------+--------+
|ID1|ID2|    date_1|    date_2|    date_3|   description|  amount|
+---+---+----------+----------+----------+--------------+--------+
|  A|101|2023-07-15|2023-07-15|2023-07-15|QLD-individual|130000.0|
+---+---+----------+----------+----------+--------------+--------+

